### Analyzing the Effects of Top/Bottom Coding on The Accuracy of Exponential Smoothing Forecasts

***

***

## Import Modules

Note the functions imported from `helper_functions`, as these are custom functions written by the paper authors. See `helper_functions.py` for comments and functions descriptions.

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer, Detrender

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import full_coding_analysis

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

Import data.

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [6]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [7]:
results_dict_ses

{'h=1, Top 0.1': {'Mean Accuracies': array([55.3207, 55.3207]),
  'Protected Mean Accuracies:': array([101.5816, 101.5816]),
  '% Change Mean accuracy:': array([-0.8362, -0.8362]),
  '% Change Median accuracy:': array([-1.7635, -1.7635]),
  '% Forecasted Points adjusted downward:': 0.7317,
  '% Forecasted Points adjusted upward:': 0.1951,
  '% Series with improved accuracy:': array([0.311, 0.311]),
  '% Series with reduced accuracy:': array([0.6159, 0.6159]),
  'Original Mean Absolute Error Upward Adjusted:': 67.8974,
  'Original Mean Absolute Error Downward Adjusted:': 55.5374,
  'Protected Mean Absolute Error Upward Adjusted:': 67.0723,
  'Protected Mean Absolute Error Downward Adjusted:': 118.9807},
 'h=20, Top 0.1': {'Mean Accuracies': array([112.5495, 135.001 ]),
  'Protected Mean Accuracies:': array([146.3865, 168.0854]),
  '% Change Mean accuracy:': array([-0.3006, -0.2451]),
  '% Change Median accuracy:': array([-0.4025, -0.3255]),
  '% Forecasted Points adjusted downward:': 0.

In [8]:
original_forecasts = fcasts_ses['h=20, Bottom 0.1']
protected_forecasts = fcasts_protected_ses['h=20, Bottom 0.1']
test = tests['h=20, Bottom 0.1']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.34554140127388533

In [13]:
np.mean(adjusted_up[improved])

0.6544585987261147

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [16]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [17]:
results_dict_des

{'h=1, Top 0.1': {'Mean Accuracies': array([54.4492, 54.4492]),
  'Protected Mean Accuracies:': array([98.3375, 98.3375]),
  '% Change Mean accuracy:': array([-0.806, -0.806]),
  '% Change Median accuracy:': array([-2.1624, -2.1624]),
  '% Forecasted Points adjusted downward:': 0.7561,
  '% Forecasted Points adjusted upward:': 0.2439,
  '% Series with improved accuracy:': array([0.4024, 0.4024]),
  '% Series with reduced accuracy:': array([0.5976, 0.5976]),
  'Original Mean Absolute Error Upward Adjusted:': 66.1507,
  'Original Mean Absolute Error Downward Adjusted:': 50.6745,
  'Protected Mean Absolute Error Upward Adjusted:': 65.1058,
  'Protected Mean Absolute Error Downward Adjusted:': 109.0574},
 'h=20, Top 0.1': {'Mean Accuracies': array([109.1455, 132.0211]),
  'Protected Mean Accuracies:': array([130.8415, 151.9802]),
  '% Change Mean accuracy:': array([-0.1988, -0.1512]),
  '% Change Median accuracy:': array([-0.2298, -0.1359]),
  '% Forecasted Points adjusted downward:': 0.78

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [20]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [21]:
results_dict_tes

{'h=1, Top 0.1': {'Mean Accuracies': array([66.4076, 66.4076]),
  'Protected Mean Accuracies:': array([110.1007, 110.1007]),
  '% Change Mean accuracy:': array([-0.658, -0.658]),
  '% Change Median accuracy:': array([-1.1204, -1.1204]),
  '% Forecasted Points adjusted downward:': 0.7988,
  '% Forecasted Points adjusted upward:': 0.2012,
  '% Series with improved accuracy:': array([0.3476, 0.3476]),
  '% Series with reduced accuracy:': array([0.6524, 0.6524]),
  'Original Mean Absolute Error Upward Adjusted:': 65.6984,
  'Original Mean Absolute Error Downward Adjusted:': 66.5863,
  'Protected Mean Absolute Error Upward Adjusted:': 64.4645,
  'Protected Mean Absolute Error Downward Adjusted:': 121.5968},
 'h=20, Top 0.1': {'Mean Accuracies': array([116.7834, 138.5197]),
  'Protected Mean Accuracies:': array([138.4417, 157.6612]),
  '% Change Mean accuracy:': array([-0.1855, -0.1382]),
  '% Change Median accuracy:': array([-0.2449, -0.2277]),
  '% Forecasted Points adjusted downward:': 0.